In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
import pygeos
import osmnx as ox
import shapely as shp
from ast import literal_eval
from pyproj import Geod
import matplotlib.pyplot as plt
import folium

Let’s use a newer dataset https://health.data.ny.gov/Health/Institutional-Provider-Network-Data-2021-Quarter-1/tfz4-6xw3 

I think each record represents a physician accepting different healthcare insurance. 

1. Find records with the following criteria.
    - ‘Designated Service’: 250 (Emergency Medication) or 823 (Urgent Care) 
    - or ‘Site Name’ has Emergency or Urgent (regardless of Designated Service)
2. Count the number of providers per facility using ‘NPI’ and ‘Site Name’
    - Exclude ‘NPI’ values 9999999999 & 8888888888 or some other invalid values
    - Exclude ‘Site Name’ is NaN. 
3. Final products
    - Doctor counts per facility: Hospital_ID, Address, Lat, Long, ProviderCount
    - List of Doctors (NPI) associated with each facility: Hospital_ID, Address, NPI


In [3]:
# Census tracts in NYC 
tracts = gpd.read_file('./data/new_york_census_tract.geojson')
tracts['GEOID_County'] = tracts.apply(lambda x:x['GEOID'][0:5], axis=1)
tracts.head(3)

,GEOID,area_sqkm,geometry,GEOID_County
0,36081003400,0.213985,"POLYGON ((311370.732 57391.499, 311384.757 573...",36081
1,36081003900,0.146969,"POLYGON ((304857.478 66110.315, 304948.710 663...",36081
2,36081003600,0.180006,"POLYGON ((311522.128 57288.347, 311596.477 573...",36081


In [4]:
# Provider data 
providers = pd.read_csv("data/Institutional_Provider_Network_Data__2021_Quarter_1.csv")
providers = gpd.GeoDataFrame(providers, geometry=gpd.points_from_xy(providers['Longitude'], providers['Latitude'], crs="EPSG:4326"))
providers['Designated Service'] = providers['Designated Service'].apply(lambda x: '{0:0>3}'.format(x))
providers = providers.to_crs(tracts.crs)

providers.head(3)

/var/folders/dv/bcrkszqj0634wz_17m9xw0qr0000gn/T/ipykernel_26482/779929108.py:2: DtypeWarning: Columns (14,15,16,80) have mixed types. Specify dtype option on import or set low_memory=False.
  providers = pd.read_csv("data/Institutional_Provider_Network_Data__2021_Quarter_1.csv")


,Site Name,Room/Suite,Address,Town/City,CountyFIPS,County Name,State,Zip Code,Latitude,Longitude,...,Servicing County 3,Servicing County 4,Servicing County 5,OASAS Program Reporting Unit Number,OASAS Provider Number,OMH ID,Plan Name,Provider Network Data System Identification Number,Quarter/Year,geometry
0,02 SOLUTIONS,NaN,1406B ROUTE 9,CLIFTON PARK,36091.0,Saratoga,NY,12065.0,42.825080,-73.733701,...,35,39,57,NaN,NaN,NaN,Nascentia Health Options,91,Quarter 1 2021,POINT (321788.653 295329.915)
1,02 SOLUTIONS,NaN,1406B ROUTE 9,CLIFTON PARK,36091.0,Saratoga,NY,12065.0,42.825080,-73.733701,...,113,115,999,NaN,NaN,NaN,Nascentia Health Options,91,Quarter 1 2021,POINT (321788.653 295329.915)
2,1 BETHESDA DRIVE OPERATING COMPANY LLC DB ELDE...,NaN,1 BETHESDA DR,HORNELL,36101.0,Steuben,NY,14843.0,42.346198,-77.665526,...,999,999,999,NaN,NaN,NaN,New York Quality Healthcare Corporation,44,Quarter 1 2021,POINT (-2053.454 248396.682)


In [5]:
providers

,Site Name,Room/Suite,Address,Town/City,CountyFIPS,County Name,State,Zip Code,Latitude,Longitude,...,Servicing County 3,Servicing County 4,Servicing County 5,OASAS Program Reporting Unit Number,OASAS Provider Number,OMH ID,Plan Name,Provider Network Data System Identification Number,Quarter/Year,geometry
0,02 SOLUTIONS,NaN,1406B ROUTE 9,CLIFTON PARK,36091.0,Saratoga,NY,12065.0,42.825080,-73.733701,...,35,39,57,NaN,NaN,NaN,Nascentia Health Options,91,Quarter 1 2021,POINT (321788.653 295329.915)
1,02 SOLUTIONS,NaN,1406B ROUTE 9,CLIFTON PARK,36091.0,Saratoga,NY,12065.0,42.825080,-73.733701,...,113,115,999,NaN,NaN,NaN,Nascentia Health Options,91,Quarter 1 2021,POINT (321788.653 295329.915)
2,1 BETHESDA DRIVE OPERATING COMPANY LLC DB ELDE...,NaN,1 BETHESDA DR,HORNELL,36101.0,Steuben,NY,14843.0,42.346198,-77.665526,...,999,999,999,NaN,NaN,NaN,New York Quality Healthcare Corporation,44,Quarter 1 2021,POINT (-2053.454 248396.682)
3,1 BETHESDA DRIVE OPERATING COMPANY LLC DB ELDE...,NaN,1 BETHESDA DR,HORNELL,36101.0,Steuben,NY,14843.0,42.346198,-77.665526,...,999,999,999,NaN,NaN,NaN,New York Quality Healthcare Corporation,44,Quarter 1 2021,POINT (-2053.454 248396.682)
4,1 BETHESDA DRIVE OPERATING COMPANY LLC DB ELDE...,NaN,1 BETHESDA DR,HORNELL,36101.0,Steuben,NY,14843.0,42.346198,-77.665526,...,999,999,999,NaN,NaN,NaN,New York Quality Healthcare Corporation,44,Quarter 1 2021,POINT (-2053.454 248396.682)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570807,ZWH MEDICAL CARE PC,NaN,6417 BROADWAY,WOODSIDE,36081.0,Queens,NY,11377.0,40.749624,-73.897965,...,999,999,999,NaN,NaN,NaN,"Affinity Health Plan, Inc.",6,Quarter 1 2021,POINT (308617.100 64740.139)
570808,ZYCARE MEDICAL ASSOCIATES PLLC,Suite CS 8,13338 41ST RD,FLUSHING,36081.0,Queens,NY,11355.0,40.756774,-73.830378,...,999,999,999,NaN,NaN,NaN,"Affinity Health Plan, Inc.",6,Quarter 1 2021,POINT (314323.456 65542.994)
570809,"ZYGMUNT DOROZYNSKI, DDS FAMILY DENTISTRY PC",NaN,130 N FRONT ST,KINGSTON,36111.0,Ulster,NY,12401.0,41.934896,-74.024026,...,999,999,999,NaN,NaN,NaN,Empire BC/ Empire BCBS/ Empire HealthChoice HM...,57,Quarter 1 2021,POINT (298007.045 196378.830)
570810,"ZYGMUNT DOROZYNSKI, DDS FAMILY DENTISTRY PC",NaN,130 N FRONT ST,KINGSTON,36111.0,Ulster,NY,12401.0,41.934896,-74.024026,...,999,999,999,NaN,NaN,NaN,"Empire Healthchoice Assurance, Inc",100,Quarter 1 2021,POINT (298007.045 196378.830)


In [6]:
# Provider wiithin NYC
tract_geom_union = pygeos.from_shapely(tracts['geometry'].unary_union)

provider_geom = pygeos.from_shapely(providers['geometry'].values)    
provider_geom_tree = pygeos.STRtree(provider_geom, leafsize=50)

query_intersect = provider_geom_tree.query(tract_geom_union, predicate='contains')
providers_nyc = providers.loc[query_intersect]
providers_nyc.head(3)

,Site Name,Room/Suite,Address,Town/City,CountyFIPS,County Name,State,Zip Code,Latitude,Longitude,...,Servicing County 3,Servicing County 4,Servicing County 5,OASAS Program Reporting Unit Number,OASAS Provider Number,OMH ID,Plan Name,Provider Network Data System Identification Number,Quarter/Year,geometry
1029,210 MJD PRECISION URGENT CARE,NaN,1737 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751462,...,999,999,999,NaN,NaN,NaN,Consolidated Health Plans: MagnaCare,169,Quarter 1 2021,POINT (321038.397 47583.142)
463720,SOUTH ISLAND MED ASSOCIATES,NaN,1735 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751460,...,999,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (321038.567 47583.143)
414968,RENAL DIVISION OF MT. SINAI,NaN,1731 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751456,...,999,999,999,NaN,NaN,NaN,Oscar Insurance Corp.,75,Quarter 1 2021,POINT (321038.905 47583.144)


In [7]:
providers_nyc

,Site Name,Room/Suite,Address,Town/City,CountyFIPS,County Name,State,Zip Code,Latitude,Longitude,...,Servicing County 3,Servicing County 4,Servicing County 5,OASAS Program Reporting Unit Number,OASAS Provider Number,OMH ID,Plan Name,Provider Network Data System Identification Number,Quarter/Year,geometry
1029,210 MJD PRECISION URGENT CARE,NaN,1737 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751462,...,999,999,999,NaN,NaN,NaN,Consolidated Health Plans: MagnaCare,169,Quarter 1 2021,POINT (321038.397 47583.142)
463720,SOUTH ISLAND MED ASSOCIATES,NaN,1735 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751460,...,999,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (321038.567 47583.143)
414968,RENAL DIVISION OF MT. SINAI,NaN,1731 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751456,...,999,999,999,NaN,NaN,NaN,Oscar Insurance Corp.,75,Quarter 1 2021,POINT (321038.905 47583.144)
329494,NEW HEALTH ASSOCIATES,NaN,1731 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751456,...,999,999,999,NaN,NaN,NaN,Nippon Life Insurance Company of America: Mult...,111,Quarter 1 2021,POINT (321038.905 47583.144)
319789,MOUNT SINAI HOSPITAL-PROF FEES,NaN,1731 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751456,...,999,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (321038.905 47583.144)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23211,"ALPINE HOME HEALTH CARE, LLC",NaN,4770 WHITE PLAINS RD,BRONX,36005.0,Bronx,NY,10470.0,40.905224,-73.849340,...,81,61,85,NaN,NaN,NaN,"MetroPlus Health Plan, Inc.",66,Quarter 1 2021,POINT (312693.878 82025.403)
23212,"ALPINE HOME HEALTH CARE, LLC",NaN,4770 WHITE PLAINS RD,BRONX,36005.0,Bronx,NY,10470.0,40.905224,-73.849340,...,81,61,85,NaN,NaN,NaN,"MetroPlus Health Plan, Inc.",66,Quarter 1 2021,POINT (312693.878 82025.403)
23216,"ALPINE HOME HEALTH CARE, LLC",NaN,4770 WHITE PLAINS RD,BRONX,36005.0,Bronx,NY,10470.0,40.905224,-73.849340,...,81,61,85,NaN,NaN,NaN,"MetroPlus Health Plan, Inc.",66,Quarter 1 2021,POINT (312693.878 82025.403)
82571,CATHOLIC GUARDIAN SERVICES,NaN,743 E 243RD ST,BRONX,36005.0,Bronx,NY,10470.0,40.906027,-73.847213,...,999,999,999,NaN,NaN,NaN,SelectHealth SNP,126,Quarter 1 2021,POINT (312872.933 82114.888)


In [5]:
providers_nyc['NPI'].value_counts()

9999999999    6807
8888888888    4554
1821036807    1985
1366430563    1716
1831269315    1596
              ... 
1780121228       1
1972042901       1
1669710117       1
1073721130       1
1558532796       1
Name: NPI, Length: 11436, dtype: int64

In [6]:
providers_nyc.loc[providers_nyc['NPI'] == 1821036807]

,Site Name,Room/Suite,Address,Town/City,CountyFIPS,County Name,State,Zip Code,Latitude,Longitude,...,Servicing County 3,Servicing County 4,Servicing County 5,OASAS Program Reporting Unit Number,OASAS Provider Number,OMH ID,Plan Name,Provider Network Data System Identification Number,Quarter/Year,geometry
273345,LENOX HILL RADIO MED IMAGING ASSO PC,NaN,2832 LINDEN BLVD,BROOKLYN,36047.0,Kings,NY,11208.0,40.669604,-73.859551,...,999,999,999,NaN,NaN,NaN,"Healthfirst Health Plan, Inc. / Healthfirst PH...",52,Quarter 1 2021,POINT (311875.518 55858.569)
274334,LENOX HILL RADIOLOGY & MEDICAL IMAGING ASSOCIATES,NaN,2832 LINDEN BLVD,BROOKLYN,36047.0,Kings,NY,11208.0,40.669604,-73.859551,...,999,999,999,NaN,NaN,NaN,Partners Health Plan,78,Quarter 1 2021,POINT (311875.518 55858.569)
273563,LENOX HILL RADIOLOGY,NaN,2832 LINDEN BLVD,BROOKLYN,36047.0,Kings,NY,11208.0,40.669604,-73.859551,...,999,999,999,NaN,NaN,NaN,Partners Health Plan,78,Quarter 1 2021,POINT (311875.518 55858.569)
273564,LENOX HILL RADIOLOGY,NaN,2832 LINDEN BLVD,BROOKLYN,36047.0,Kings,NY,11208.0,40.669604,-73.859551,...,999,999,999,NaN,NaN,NaN,Empire BC/ Empire BCBS/ Empire HealthChoice HM...,57,Quarter 1 2021,POINT (311875.518 55858.569)
275793,LENOX HILL RADIOLOGY MEDICAL IMAGING A,PHP-LINDENWOOD,2832 LINDEN BLVD,BROOKLYN,36047.0,Kings,NY,11208.0,40.669604,-73.859551,...,999,999,999,NaN,NaN,NaN,VillageCareMAX,90,Quarter 1 2021,POINT (311875.518 55858.569)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274206,LENOX HILL RADIOLOGY - WASHINGTON HEIGHTS NORTH,NaN,4334 BROADWAY,NEW YORK,36061.0,New York,NY,10033.0,40.852974,-73.934110,...,999,999,999,NaN,NaN,NaN,Delaware American Life Insurance Company: PHCS,142,Quarter 1 2021,POINT (305555.925 76214.155)
274725,LENOX HILL RADIOLOGY & MEDICAL IMAGING ASSOCIA...,LHR of Washington He,4334 BROADWAY,NEW YORK,36061.0,New York,NY,10033.0,40.852974,-73.934110,...,999,999,999,NaN,NaN,NaN,"MetroPlus Health Plan, Inc.",66,Quarter 1 2021,POINT (305555.925 76214.155)
274726,LENOX HILL RADIOLOGY & MEDICAL IMAGING ASSOCIA...,LHR of Washington He,4334 BROADWAY,NEW YORK,36061.0,New York,NY,10033.0,40.852974,-73.934110,...,999,999,999,NaN,NaN,NaN,"MetroPlus Health Plan, Inc. (HIV/SNP)",95,Quarter 1 2021,POINT (305555.925 76214.155)
278457,LHR OF WASHINGTON HEIGHTS NORTH,STE 1,4334 BROADWAY,NEW YORK,36061.0,New York,NY,10033.0,40.852974,-73.934110,...,999,999,999,NaN,NaN,NaN,Partners Health Plan,78,Quarter 1 2021,POINT (305555.925 76214.155)


In [7]:
providers_nyc['Designated Service'].value_counts()

760    84150
914    36569
375    13320
307     7725
911     6958
       ...  
835        1
205        1
084        1
915        1
981        1
Name: Designated Service, Length: 259, dtype: int64

In [8]:
# Exclude invalid values
providers_nyc = providers_nyc.loc[~providers_nyc['Site Name'].isna()]
providers_nyc

,Site Name,Room/Suite,Address,Town/City,CountyFIPS,County Name,State,Zip Code,Latitude,Longitude,...,Servicing County 3,Servicing County 4,Servicing County 5,OASAS Program Reporting Unit Number,OASAS Provider Number,OMH ID,Plan Name,Provider Network Data System Identification Number,Quarter/Year,geometry
1029,210 MJD PRECISION URGENT CARE,NaN,1737 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751462,...,999,999,999,NaN,NaN,NaN,Consolidated Health Plans: MagnaCare,169,Quarter 1 2021,POINT (321038.397 47583.142)
463720,SOUTH ISLAND MED ASSOCIATES,NaN,1735 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751460,...,999,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (321038.567 47583.143)
414968,RENAL DIVISION OF MT. SINAI,NaN,1731 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751456,...,999,999,999,NaN,NaN,NaN,Oscar Insurance Corp.,75,Quarter 1 2021,POINT (321038.905 47583.144)
329494,NEW HEALTH ASSOCIATES,NaN,1731 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751456,...,999,999,999,NaN,NaN,NaN,Nippon Life Insurance Company of America: Mult...,111,Quarter 1 2021,POINT (321038.905 47583.144)
319789,MOUNT SINAI HOSPITAL-PROF FEES,NaN,1731 SEAGIRT BLVD,FAR ROCKAWAY,36081.0,Queens,NY,11691.0,40.594899,-73.751456,...,999,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (321038.905 47583.144)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23211,"ALPINE HOME HEALTH CARE, LLC",NaN,4770 WHITE PLAINS RD,BRONX,36005.0,Bronx,NY,10470.0,40.905224,-73.849340,...,81,61,85,NaN,NaN,NaN,"MetroPlus Health Plan, Inc.",66,Quarter 1 2021,POINT (312693.878 82025.403)
23212,"ALPINE HOME HEALTH CARE, LLC",NaN,4770 WHITE PLAINS RD,BRONX,36005.0,Bronx,NY,10470.0,40.905224,-73.849340,...,81,61,85,NaN,NaN,NaN,"MetroPlus Health Plan, Inc.",66,Quarter 1 2021,POINT (312693.878 82025.403)
23216,"ALPINE HOME HEALTH CARE, LLC",NaN,4770 WHITE PLAINS RD,BRONX,36005.0,Bronx,NY,10470.0,40.905224,-73.849340,...,81,61,85,NaN,NaN,NaN,"MetroPlus Health Plan, Inc.",66,Quarter 1 2021,POINT (312693.878 82025.403)
82571,CATHOLIC GUARDIAN SERVICES,NaN,743 E 243RD ST,BRONX,36005.0,Bronx,NY,10470.0,40.906027,-73.847213,...,999,999,999,NaN,NaN,NaN,SelectHealth SNP,126,Quarter 1 2021,POINT (312872.933 82114.888)


In [9]:
# You probably can do something like this
unique_df = pd.DataFrame(providers_nyc[['Site Name', 'Address', 'NPI']].value_counts().reset_index())
unique_df

,Site Name,Address,NPI,0
0,THE DORAL INVESTORS GROUP LLC DBA HOUSE CALLS ...,2440 FULTON ST,1639473580,500
1,"PEOPLE CARE, INC.",116 W 32ND ST,1841302445,373
2,ELMHURST HOSPITAL CENTER,7901 BROADWAY,1063426377,229
3,THE BRIDGE,248 W 108TH ST,1831269315,223
4,NEW HORIZON COUNSELING CENTER,718 BEACH 20TH ST,1366430563,203
...,...,...,...,...
53110,KUMP & SAYEGH FAMILY MEDICAL SVCS PC,530 W 236TH ST,1336316348,1
53111,KUMAR PHYSICIAN OFFICE,25012B HILLSIDE AVE,1447628342,1
53112,"KUMAR MEDICAL CARE, P.C.",2310 30TH AVE,1548601024,1
53113,"KUMAR MEDICAL CARE, P.C.",12415 JAMAICA AVE,1548601024,1


## Notes



benefits over health facility genearl information:

- more complete (e.g., has urgent care)
- have more in-depth segmentation (instead of clinic, have type of clinic and what it actually does, which can inform whether a facility supports walk-ins), (desser + service types)
    - e.g., hospital extension clinics in hfgi include a methadone center, a dance injury center, and a dentistry center while in pnds, they are segmented into more identifying categories
- includes the number of providers (noprov)

cons versus health facility general information

- more complex (has a bunch of random, irrelevant stuff)
- indexed by health facilities & plans based on which plans contract with what facilities -- not directly comparable to healthcare accessibility without some tinkering

documentation: https://www.health.ny.gov/health_care/managed_care/docs/dictionary.pdf

what columns mean: https://www.opendatanetwork.com/dataset/health.data.ny.gov/eary-k6ia

To use this data, we need to assume the following:

1. The institutions that are involved in group medical insurance plans are representative of all relevant institutions.
    - this is somewhat reasonable since these institutions would be the most accessible to the average person
2. The assigned designated service is representative of what a facility actually does and can be used to determine whether a facility supports walk-ins or not
    - (can't guarantee that walk-ins are supported but can make generalizations like that clinics generally allow walk-ins)
3. Some subset of the data can be validly excluded, as it is not relevant to medical accessibility
    - should ideally exclude all medically tangential businesses while including relatively relevant ones (like urgent care)

Potential service codes to look for (that might support walk-ins)

- 005 state operated clinic
- 011 general hospital
- 034 state operated facility
- 150 pediatrics
- 252 primary care initiative in underserved areas
- 760 pharmacy?
- 776 general mpractice only - nospec?
- 777 all physician?
- 823 urgent care
- 899 hospital inpatient?
- 901 emergency room
- 914 general medicine - clinic speciality ?
- 978 preferred primary care clinic?
- clinic specialty things (have basically one for every field)